In [37]:
import random
import time

In [38]:
class Share:
    # declaring company name, last traded price, previous day last trade price for each share
    def __init__(self, company_name, last_traded_price, prev_day_last_trade):
        self.company_name = company_name
        self.bids = []
        # bids array stores the bids for that particular share
        self.asks = []
        # asks array stores the asks for that particular share
        self.last_traded_price = last_traded_price
        self.prev_day_last_trade = prev_day_last_trade
        self.last_trade_time = time.localtime()

In [39]:
class StockExchange:
    def __init__(self):
        self.shares = []
        self.ome = self.OrderMatchingEngine()
    
    def add_share(self, share):
        self.shares.append(share)
        
    # function to retrieve the share corresponding to given company
    def get_share(self, company_name):
        for share in self.shares:
            if share.company_name == company_name:
                return share
        return None
    
    # method to accept bid
    def bid(self, bid_price, number, company_name, trader, stock_exchange):
        share = self.get_share(company_name)
        if not share:
            return None
        
        t = time.localtime()
        share.bids.append([bid_price, t, number, trader])
        # bids are sorted in descending order of bid price, and ascending order of time according to price-time priority
        share.bids = sorted(share.bids, key = lambda x: (-x[0],x[1]))
        
        # if the bid is not in top 5, no action is performed
        checker = self.ome.check_bid_position(bid_price, number, t, trader, share)
        if(checker >= 5) :
            print(f'Bid of price Rs. {bid_price} is not in top 5!')
            return None
        print(f'{trader.name} bidding {number} shares of {company_name} at Rs. {bid_price}')

        rem_number = 0
        # invoking function to check for a trade (if possible), and returning the remaining number of shares (useful in case of partial trade) 
        rem_number = self.ome.accepted_bid(bid_price, number, company_name, trader, stock_exchange)
        if(rem_number == None):
            return
        if rem_number > 0 :
            # partial trade done
            share.bids.remove([bid_price, t, number, trader])
            share.bids.append([bid_price, t, rem_number, trader])
            share.bids = sorted(share.bids, key = lambda x: (-x[0],x[1]))
        if rem_number == 0 :
            # all shares traded
            share.bids.remove([bid_price, t, number, trader])
    
    # method to accept ask
    def ask(self, ask_price, number, company_name, trader, stock_exchange):
        share = self.get_share(company_name)
        if not share:
            return None
        
        t = time.localtime()
        share.asks.append([ask_price, t, number, trader])
        # asks are sorted in ascending order of ask price, and ascending order of time according to price-time priority
        share.asks = sorted(share.asks, key = lambda x: (x[0],x[1]))
        
        # if the ask is not in top 5, no action is performed
        checker = self.ome.check_ask_position(ask_price, number, t, trader, share)
        if(checker >= 5) :
            print(f'Ask of price Rs. {ask_price} is not in top 5!')
            return None
        print(f'{trader.name} selling {number} shares of {company_name} at Rs. {ask_price}')

        rem_number = 0
        # invoking function to check for a trade (if possible), and returning the remaining number of shares (useful in case of partial trade) 
        rem_number = self.ome.accepted_ask(ask_price, number, company_name, trader, stock_exchange)

        if(rem_number == None):
            return
        if rem_number > 0 :
            # partial trade done
            share.asks.remove([ask_price, t, number, trader])
            share.asks.append([ask_price, t, rem_number, trader])
            share.asks = sorted(share.asks, key = lambda x: (x[0],x[1]))
        if rem_number == 0 :
            # all shares traded
            share.asks.remove([ask_price, t, number, trader])
    
    # function for printing last traded price
    def last_traded_price(self):
        for share in self.shares:
            print(f"{share.company_name}: {share.last_traded_price}")
    
    # function to get the best bid and ask for a share
    def get_best_bid_ask(self, company_name):
        share = self.get_share(company_name)
        if not share:
            print('Share not found!')
            return None
        if not share.bids and not share.asks:
            return None, None
        elif not share.bids:
            return None, share.asks[0]
        elif not share.asks:
            return share.bids[0], None
        else:
            return share.bids[0], share.asks[0]
    
    # function to get top five best bids and asks for a share
    def get_best_5_bid_ask(self, company_name):
        share = self.get_share(company_name)
        if not share:
            print('Share not found!')
            return None
        top_5_bids = []
        top_5_asks = []
        for i in range(min(5, len(share.bids))):
            top_5_bids.append(share.bids[i])
        for i in range(min(5, len(share.asks))):
            top_5_asks.append(share.asks[i])
    
    class OrderMatchingEngine :
        
        # checks if position is in top 5
        def check_bid_position(self, bid_price, number, time, trader, share):
            counter = 0
            for i in share.bids:
                if(i[0] == bid_price and i[1] == time and i[2] == number and i[3] is trader):
                    return counter
                counter += 1
            return counter
        
        # checks if position is in top 5
        def check_ask_position(self, ask_price, number, time, trader, share):
            counter = 0
            for i in share.asks:
                if(i[0] == ask_price and i[1] == time and i[2] == number and i[3] is trader):
                    return counter
                counter += 1
            return counter
        
        def accepted_bid(self, bid_price, number, company_name, trader, stock_exchange):
           
            # market time is between 09:00 hours and 15:30 hours 
            if(time.localtime().tm_hour < 9): 
                print('Cannot trade outside trading hours')
                return None
            if(time.localtime().tm_hour > 15):
                print('Cannot trade outside trading hours')
                print('TIME UP! Bids and Asks will now be empty')
                for share in self.shares: 
                    share.bids = []
                    share.asks = []
                return None
            if(time.localtime().tm_hour == 15 and time.localtime().tm_min > 30): 
                print('Cannot trade outside trading hours')
                print('TIME UP! Bids and Asks will now be empty')
                for share in self.shares: 
                    share.bids = []
                    share.asks = []
                return None
            
            share = stock_exchange.get_share(company_name)

            while len(share.asks) > 0 and number != 0 and share.asks[0][0] <= bid_price:
                # updates previous day price when the day changes
                t = time.localtime()
                if share.last_trade_time.tm_yday < t.tm_yday and share.last_trade_time.tm_yday != None:
                    share.prev_day_last_trade = share.last_traded_price
                
                # updates last traded price and time whenever a trade is matched
                share.last_traded_price = share.asks[0][0]
                share.last_trade_time = t
                
                # matching the best ask with current bid
                if share.asks[0][2] == number :
                    print(f"{trader.name} bought {number} shares of {company_name} from {share.asks[0][3].name} at price Rs. {share.asks[0][0]}")                            
                    share.asks[0][3].balance += (share.asks[0][0] * number)
                    share.asks[0][3].portfolio[company_name] -= number
                    trader.balance -= (share.asks[0][0] * number)
                    trader.portfolio[company_name] += number
                    share.asks.remove(share.asks[0])
                    number = 0
                # matching the best ask with current bid
                elif share.asks[0][2] > number :
                    print(f"{trader.name} bought {number} shares of {company_name} from {share.asks[0][3].name} at price Rs. {share.asks[0][0]}")
                    share.asks[0][3].balance += (share.asks[0][0] * number)
                    share.asks[0][3].portfolio[company_name] -= number
                    trader.balance -= (share.asks[0][0] * number)
                    trader.portfolio[company_name] += number
                    share.asks[0][2] = share.asks[0][2] - number
                    number = 0
                # matching the best ask with current bid (partial matching here) 
                else :
                    print(f"{trader.name} bought {share.asks[0][2]} shares of {company_name} from {share.asks[0][3].name} at price Rs. {share.asks[0][0]}")
                    share.asks[0][3].balance += (share.asks[0][0] * share.asks[0][2])
                    share.asks[0][3].portfolio[company_name] -= share.asks[0][2]
                    trader.balance -= (share.asks[0][0] * share.asks[0][2])
                    trader.portfolio[company_name] += share.asks[0][2]
                    number -= share.asks[0][2]
                    share.asks.remove(share.asks[0])

            return number
        
        def accepted_ask(self, ask_price, number, company_name, trader, stock_exchange):
            # market time is between 09:00 hours and 15:30 hours
            
            if(time.localtime().tm_hour < 9): 
                print('Cannot trade outside trading hours')
                return None
            if(time.localtime().tm_hour > 15):
                print('Cannot trade outside trading hours')
                print('TIME UP! Bids and Asks will now be empty')
                for share in self.shares: 
                    share.bids = []
                    share.asks = []
                return None
            if(time.localtime().tm_hour == 15 and time.localtime().tm_min > 30): 
                print('Cannot trade outside trading hours')
                print('TIME UP! Bids and Asks will now be empty')
                for share in self.shares: 
                    share.bids = []
                    share.asks = []
                return None
            
            share = stock_exchange.get_share(company_name)

            while len(share.bids) > 0 and number != 0 and share.bids[0][0] >= ask_price:
                # updates previous day price when the day changes               
                t = time.localtime()
                if share.last_trade_time.tm_yday < t.tm_yday :
                    share.prev_day_last_trade = share.last_traded_price

                # updates last traded price and time whenever a trade is matched
                share.last_traded_price = ask_price
                share.last_trade_time = t
                
                # matching the best bid with current ask
                if share.bids[0][2] == number :
                    print(f"{trader.name} sold {number} shares of {company_name} to {share.asks[0][3].name} at price Rs. {ask_price}")
                    share.bids[0][3].balance -= (ask_price * number)
                    share.bids[0][3].portfolio[company_name] += number
                    trader.balance += (ask_price * number)
                    trader.portfolio[company_name] -= number
                    share.bids.remove(share.bids[0])
                    number = 0
                # matching the best bid with current ask
                elif share.bids[0][2] > number :
                    print(f"{trader.name} sold {number} shares of {company_name} to {share.asks[0][3].name} at price Rs. {ask_price}")
                    share.bids[0][3].balance -= (ask_price * number)
                    share.bids[0][3].portfolio[company_name] += number
                    trader.balance += (ask_price * number)
                    trader.portfolio[company_name] -= number
                    share.bids[0][2] = share.bids[0][2] - number
                    number = 0
                # matching the best bid with current ask (partial matching here) 
                else :
                    print(f"{trader.name} sold {share.bids[0][2]} shares of {company_name} to {share.asks[0][3].name} at price Rs. {ask_price}")
                    share.bids[0][3].balance -= (ask_price * share.asks[0][2])
                    share.bids[0][3].portfolio[company_name] += share.asks[0][2]
                    trader.balance += (ask_price * share.asks[0][2])
                    trader.portfolio[company_name] -= share.asks[0][2]
                    number -= share.bids[0][2]
                    share.bids.remove(share.bids[0])
            return number

In [40]:
class Trader:
    # each trader has attributes name, balance and portfolio (shares and respective count) 
    def __init__(self, name, balance, portfolio):
        self.name = name
        self.balance = balance
        self.portfolio = portfolio
        self.oms = self.OrderManagementSystem()
        
    def get_share(self, company_name, stock_exchange):
        stock_exchange.get_share(company_name)
    
    # action by the trader to buy or sell stock
    # buy_or_sell is 1 if trader wants to buy or 0 if he wants to sell
    def action(self, company_name, quantity, buy_or_sell, stock_exchange, trader):
        share = stock_exchange.get_share(company_name)

        # for only a bid or an ask,choose the opposite side 
        if len(share.bids) + len(share.asks) == 1 :
            if len(share.bids):
                self.oms.sell_stock(share, quantity, trader, stock_exchange)
            else :
                self.oms.buy_stock(share, quantity, trader, stock_exchange)
        elif buy_or_sell:
            self.oms.buy_stock(share, quantity, trader, stock_exchange)
        else :
            self.oms.sell_stock(share, quantity, trader, stock_exchange)
    
    class OrderManagementSystem: 
        # method when the action is to buy
        def buy_stock(self, share, quantity, trader, stock_exchange):
            # trader can't bid if his balance is less than the total amount
            if trader.balance < share.last_traded_price * quantity:
                print(f"Sorry, you don't have enough money to buy {quantity} shares of {share.company_name}.")
                return False

            if len(share.bids) == 0 and len(share.asks) == 1 :
                trade_price = random.choice([share.last_traded_price, share.asks[0][0] * 0.95])

            elif len(share.bids) + len(share.asks) == 0 :
                trade_price = random.choice([share.prev_day_last_trade * 1.05, share.prev_day_last_trade * 0.95])
            else :
                r = random.randint(1, 3)
                if r == 1 :
                    if len(share.bids):
                        trade_price = share.bids[0][0]
                    else :
                        r = 2
                if r == 2 :
                    if len(share.asks):
                        trade_price = share.asks[0][0]
                    else :
                        trade_price = share.bids[0][0]
                else :
                    if len(share.bids) and len(share.asks):
                        trade_price = (share.asks[0][0] + share.bids[0][0]) / 2
                    elif len(share.bids):
                        trade_price = share.bids[0][0]
                    else :
                        trade_price = share.asks[0][0]
            
            stock_exchange.bid(trade_price, quantity, share.company_name, trader, stock_exchange)

        # method when the action is to sell
        def sell_stock(self, share, quantity, trader, stock_exchange):
            # if trader does not have enough shares to sell
            if share.company_name not in trader.portfolio or trader.portfolio[share.company_name] < quantity:
                print(f"Sorry, you don't have {quantity} shares of {share.company_name} to sell.")
                return False

            if len(share.bids) == 1 and len(share.asks) == 0 :
                trade_price = random.choice([share.last_traded_price, share.bids[0][0] * 1.05])
            elif len(share.bids) + len(share.asks) == 0 :
                trade_price = random.choice([share.prev_day_last_trade * 1.05, share.prev_day_last_trade * 0.95])
            else :
                r = random.randint(1, 3)
                if r == 1 :
                    if len(share.bids):
                        trade_price = share.bids[0][0]
                    else :
                        r = 2
                if r == 2 :
                    if len(share.asks):
                        trade_price = share.asks[0][0]
                    else :
                        trade_price = share.bids[0][0]
                else :
                    if len(share.bids) and len(share.asks):
                        trade_price = (share.asks[0][0] + share.bids[0][0]) / 2
                    elif len(share.bids):
                        trade_price = share.bids[0][0]
                    else :
                        trade_price = share.asks[0][0]
           
            stock_exchange.ask(trade_price, quantity, share.company_name, trader, stock_exchange)

        # display balance
        def display_balance(self, trader): 
            print(f"You have amount: Rs. {trader.balance}.")
        
        # display portfolio
        def display_portfolio(self, trader): 
            print(f"Your portfolio: {trader.portfolio}.")

In [41]:
t1 = Trader("Aastha", 20000, {'tech' : 5, 'energy' : 10} )
t2 = Trader("Divya", 25000, {'tech' : 10, 'energy' : 2} )
t3 = Trader("Tanisha",10000, {'tech' : 50, 'infra': 10, 'agri' : 2})
t4 = Trader("Shally", 35000, {} )
t5 = Trader("Ankita", 0, {'pharma' : 2, 'tech' : 20})

In [42]:
t1.oms.display_portfolio(t1)

Your portfolio: {'tech': 5, 'energy': 10}.


In [43]:
t2.oms.display_balance(t2)

You have amount: Rs. 25000.


In [44]:
A = Share("tech", 120, 120)
B = Share("energy", 3, 3)
C = Share("infra", 45, 45)
D = Share("agri", 111, 111)
E = Share("pharma", 63, 63)

In [45]:
NSE = StockExchange()
NSE.add_share(A)
NSE.add_share(B)
NSE.add_share(C)
NSE.add_share(D)
NSE.add_share(E)

In [46]:
t1.action("tech", 5, 0, NSE, t1)

Aastha selling 5 shares of tech at Rs. 114.0


In [47]:
t1.oms.display_portfolio(t1)

Your portfolio: {'tech': 5, 'energy': 10}.


In [48]:
A.asks

[[114.0,
  time.struct_time(tm_year=2023, tm_mon=3, tm_mday=13, tm_hour=15, tm_min=12, tm_sec=6, tm_wday=0, tm_yday=72, tm_isdst=0),
  5,

In [49]:
t2.action("tech", 7, 1, NSE, t2)

Divya bidding 7 shares of tech at Rs. 108.3


In [50]:
A.bids

[[108.3,
  time.struct_time(tm_year=2023, tm_mon=3, tm_mday=13, tm_hour=15, tm_min=12, tm_sec=16, tm_wday=0, tm_yday=72, tm_isdst=0),
  7,

In [51]:
t1.action("tech", 2, 0, NSE, t1)

Aastha selling 2 shares of tech at Rs. 114.0


In [52]:
A.asks

[[114.0,
  time.struct_time(tm_year=2023, tm_mon=3, tm_mday=13, tm_hour=15, tm_min=12, tm_sec=6, tm_wday=0, tm_yday=72, tm_isdst=0),
  5,
 [114.0,
  time.struct_time(tm_year=2023, tm_mon=3, tm_mday=13, tm_hour=15, tm_min=12, tm_sec=28, tm_wday=0, tm_yday=72, tm_isdst=0),
  2,

In [53]:
t3.action("infra", 2, 1, NSE, t3)

Tanisha bidding 2 shares of infra at Rs. 47.25
